In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import config

## Extract CSVs into DataFrame

In [2]:
data_science_file = "Resources/datascience_jobs_aus.csv"
data_science_df = pd.read_csv(data_science_file)
data_science_df.head()

,Job Title,Job Location,Company,Url,Estimate Base Salary,Low Estimate,High Estimate,Company Size,Company Type,Company Sector,...,cassandra_yn,hive_yn,bigml_yn,tableau_yn,powerbi_yn,nlp_yn,pytorch_yn,tensorflow_yn,mathematic_yn,statistic_yn
0,Analyst,Melbourne,ANZ Banking Group,https://www.glassdoor.com.au/partner/jobListin...,95917,80000,115000,10000+ Employees,Company - Public,Finance,...,0,0,0,0,0,0,0,0,1,0
1,Clinical Research Associate,Mulgrave,Bristol Myers Squibb,https://www.glassdoor.com.au/partner/jobListin...,96555,79000,118000,10000+ Employees,Company - Public,Pharmaceutical & Biotechnology,...,0,0,0,0,0,0,0,0,0,0
2,Clinical Research Associate,Mulgrave,Bristol Myers Squibb,https://www.glassdoor.com.au/partner/jobListin...,96555,79000,118000,10000+ Employees,Company - Public,Pharmaceutical & Biotechnology,...,0,0,0,0,0,0,0,0,0,0
3,Clinical Research Associate,Mulgrave,Bristol Myers Squibb,https://www.glassdoor.com.au/partner/jobListin...,96555,79000,118000,10000+ Employees,Company - Public,Pharmaceutical & Biotechnology,...,0,0,0,0,0,0,0,0,0,0
4,Data Scientist,Melbourne,ANZ Banking Group,https://www.glassdoor.com.au/partner/jobListin...,115631,94000,143000,10000+ Employees,Company - Public,Finance,...,0,0,0,0,0,0,0,0,0,0


In [3]:
job_listing_file = "Resources/datascience_listing.csv"
job_listing_df = pd.read_csv(job_listing_file)
job_listing_df.head()

,jobId,jobTitle,jobClassification,jobSubClassification,advertiserName,advertiserId,companyId,companyName,companyRating,listingDate,...,Haskell,Lisp,Golang,Spark,Javascript,F.,Fortran,first_seen,last_seen,recruiter
0,38098375,Senior Method Development Immunoassay Scientis...,Science & Technology,Biological & Biomedical Sciences,Hobson Prior Australia,30969882,NaN,NaN,NaN,16/01/2019 12:17,...,0,0,0,0,0,0,0,6/03/2019,11/03/2019,1
1,38259053,Pricing Data Scientist - Lower North Shore,Information & Communication Technology,Other,Randstad - Technologies,26537413,NaN,NaN,NaN,4/02/2019 13:02,...,0,0,0,0,0,0,0,6/03/2019,6/03/2019,1
2,38267899,Insights Analyst – Online Video Streaming,Information & Communication Technology,Database Development & Administration,Aurec,39407892,NaN,NaN,NaN,5/02/2019 11:33,...,0,0,0,0,0,0,0,6/03/2019,7/03/2019,1
3,38267995,Credit Risk Analyst,Banking & Financial Services,Compliance & Risk,Huxley Associates,26057722,NaN,NaN,NaN,5/02/2019 11:39,...,0,0,0,0,0,0,0,6/03/2019,7/03/2019,1
4,38269206,Data Scientist,Information & Communication Technology,Database Development & Administration,FinXL IT Professional Services,33086778,NaN,NaN,NaN,5/02/2019 12:54,...,0,0,0,0,0,0,0,6/03/2019,7/03/2019,0


## Transform data_science DataFrame

In [4]:
# Create a filtered dataframe from specific columns
data_science_cols = ["Job Location","Job Title","Estimate Base Salary"]
data_science_transformed = data_science_df[data_science_cols].copy()

# Rename the column headers
data_science_transformed= data_science_transformed.rename(columns={
                                                          "Job Location": "job_location",
                                                          "Job Title": "job_title",
                                                          "Estimate Base Salary": "salary"
                                                          })

# Clean the data by dropping duplicates and NA values and setting the index
new_data_science_transformed = data_science_transformed.dropna(how="any")
new_data_science_transformed.drop_duplicates("job_location", inplace=True)
new_data_science_transformed.head()

c:\Users\User\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,job_location,job_title,salary
0,Melbourne,Analyst,95917
1,Mulgrave,Clinical Research Associate,96555
8,Australia,Software Engineer,212000
40,Dandenong,Quality Manager,90000
44,Reservoir,Food Technologist,75000


## Transform job_listing DataFrame

In [5]:
# Create a filtered dataframe from specific columns
data_listing_cols = ["city","jobClassification"]
data_listing_transformed = job_listing_df[data_listing_cols].copy()

# Rename the column headers
data_listing_transformed= data_listing_transformed.rename(columns={"city": "job_location",
                                                          "jobClassification": "job_class"
                                                              })

# Clean the data by dropping duplicates and NA values and setting the index
new_data_listing_transformed = data_listing_transformed.dropna(how="any")
new_data_listing_transformed.drop_duplicates("job_location", inplace=True)
new_data_listing_transformed.head()


,job_location,job_class
0,Sydney,Science & Technology
1,ACT,Information & Communication Technology
8,Melbourne,Information & Communication Technology
15,Perth,Information & Communication Technology
24,Brisbane,"Mining, Resources & Energy"


## Connect to local database

In [7]:
protocol = 'postgresql'
username = config.Username
password = config.Password
host = 'localhost'
port = 5432
database_name = 'datascience_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

## Inspect Tables

In [8]:
insp.get_table_names()

['datascience', 'joblisting']

## Load

In [9]:
new_data_science_transformed.to_sql(name='datascience', con=engine, if_exists='append', index=False)

In [10]:
new_data_listing_transformed.to_sql(name='joblisting', con=engine, if_exists='append', index=False)

## Confirm data has been added by querying the datascience table

In [11]:
pd.read_sql_query('select * from datascience', con=engine).head()

,job_location,job_title,salary
0,Melbourne,Analyst,95917
1,Mulgrave,Clinical Research Associate,96555
2,Australia,Software Engineer,212000
3,Dandenong,Quality Manager,90000
4,Reservoir,Food Technologist,75000


## Confirm data has been added by querying the job listing table

In [12]:
pd.read_sql_query('select * from joblisting', con=engine).head()

,job_location,job_class
0,Sydney,Science & Technology
1,ACT,Information & Communication Technology
2,Melbourne,Information & Communication Technology
3,Perth,Information & Communication Technology
4,Brisbane,"Mining, Resources & Energy"


## Join Data from datascience and joblisting table

In [13]:
sql_join = r"""SELECT datascience.job_location, datascience.job_title, datascience.salary,joblisting.job_class
FROM datascience
INNER JOIN joblisting
ON datascience.job_location = joblisting.job_location"""
pd.read_sql_query(sql_join, con=engine).head(20)

,job_location,job_title,salary,job_class
0,Sydney,Data Scientist,125000,Science & Technology
1,Melbourne,Analyst,95917,Information & Communication Technology
2,Perth,Entry Level Media Coordinator,65520,Information & Communication Technology
3,Brisbane,Graduate Data Scientist,128589,"Mining, Resources & Energy"
4,Darwin,Data Manager,82988,Science & Technology
5,Adelaide,Software Engineer,85000,Science & Technology
6,Hobart,Biostatistician,106500,Science & Technology
7,Gold Coast,Data Engineer,82171,Information & Communication Technology
